# Initialization

In [1]:
from logging import ERROR
import os
from pathlib import Path
import signal
from subprocess import CalledProcessError
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces
from pyboolnet import log
log.setLevel(ERROR)

# mpbn init
import mpbn

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

INFO Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO NumExpr defaulting to 8 threads.


<Handlers.SIG_DFL: 0>

In [2]:
# compute trapspaces using PyBoolNet
def bench_pyboolnet(models, timeout=180, number=1, max_output=1000):
    for model in models:
        print(model)        
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                primes = bnet2primes(model + ".bnet")
                tspaces = compute_trap_spaces(primes, "min", max_output=max_output)
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except (TimeOutException, CalledProcessError, Exception):
                total_time += timeout
            except MemoryError:
                print ("MemoryError")
                total_time += timeout
            signal.alarm(0)
        
        os.system("killall -9 BNetToPrime_linux64")
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [3]:
# compute trapspaces using mpbn
def bench_mpbn(models, timeout=180, number=1, max_output=1000):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                mbn = mpbn.load(model + ".bnet")
                tspaces = list(mbn.attractors(limit=max_output))
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except AssertionError:
                if i == 0:
                    print("Non-monotonic")
            except TimeOutException:
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [4]:
# compute trapspaces using trappist
def bench_trappist(models, timeout=180, number=1, max_output=1000, **kwargs):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                tspaces = trappist.compute_trap_spaces(model + ".bnet", max_output=max_output, time_limit=timeout, **kwargs)
                nb_tspaces = len(list(tspaces))
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except (TimeOutException, CalledProcessError):
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

## PyBoolNet repository

Unfortunately all models are very easy

In [16]:
models = [str(Path().absolute() / "TCS-data-set/PyBoolNet" / name[:-5]) for name in os.listdir("TCS-data-set/PyBoolNet") if name.endswith(".bnet")]

In [13]:
bench_pyboolnet(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/saadatpour_guardcell
1 trap spaces 0.06
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/n3s1c1a
2 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/dinwoodie_stomatal
1 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/raf
2 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n7k3
10 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/klamt_tcr
8 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/zhang_tlgl
156 trap spaces 0.09
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n15k3
3 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/xiao_wnt5a
4 trap spaces 0.01
===
/home/giang-trinh/

In [12]:
bench_mpbn(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/saadatpour_guardcell
1 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/n3s1c1a
2 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/dinwoodie_stomatal
1 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/raf
2 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n7k3
Non-monotonic
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/klamt_tcr
8 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/zhang_tlgl
156 trap spaces 0.09
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n15k3
Non-monotonic
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/xiao_wnt5a
4 trap spaces 0.00
===
/home/giang-trinh/git

In [6]:
bench_trappist(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/saadatpour_guardcell
1 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/n3s1c1a
2 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/dinwoodie_stomatal
1 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/raf
2 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n7k3
10 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/klamt_tcr
8 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/zhang_tlgl
156 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n15k3
3 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/xiao_wnt5a
4 trap spaces 0.01
===
/home/giang-trinh/

In [6]:
bench_trappist(models, method='sat')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/saadatpour_guardcell
1 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/n3s1c1a
2 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/dinwoodie_stomatal
1 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/raf
2 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n7k3
10 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/klamt_tcr
8 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/zhang_tlgl
156 trap spaces 0.05
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n15k3
3 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/xiao_wnt5a
4 trap spaces 0.00
===
/home/giang-trinh/

In [6]:
bench_trappist(models, method='cp')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/saadatpour_guardcell
1 trap spaces 0.53
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/n3s1c1a
2 trap spaces 0.63
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/dinwoodie_stomatal
1 trap spaces 0.39
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/raf
2 trap spaces 0.55
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n7k3
10 trap spaces 2.07
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/klamt_tcr
8 trap spaces 1.98
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/zhang_tlgl
156 trap spaces 37.26
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n15k3
3 trap spaces 0.77
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/xiao_wnt5a
4 trap spaces 1.00
===
/home/giang-trinh

In [6]:
bench_trappist(models, method='ilp')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/saadatpour_guardcell
1 trap spaces 0.45
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/n3s1c1a
2 trap spaces 0.49
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/dinwoodie_stomatal
1 trap spaces 0.29
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/raf
2 trap spaces 0.39
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n7k3
10 trap spaces 1.46
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/klamt_tcr
8 trap spaces 1.22
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/zhang_tlgl
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/randomnet_n15k3
3 trap spaces 0.67
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/PyBoolNet/xiao_wnt5a
4 trap spaces 0.89
===
/home/giang-trinh/git_repo/